
```
# This is formatted as code
import os
import tweepy as tw
import csv
import json
import datetime
import jsonpickle
import csv

consumer_key = 'hGOrUWoKSViI11fSGNTkoGiHD'
consumer_secret = 'gyAHSYn03WlCf6pVmBJcs0wqvXIeCKGv7Q9B28gJGIb5sMulsA'
access_token = '1198651355130454016-DeyVvfOmL0FM9Aqk2KDU1c9adlTjYA'
access_token_secret = 'SDNwpW4iI2L57G59nkfeU8gumc7KuluAXU9svG9xx7vLR'

auth = tw.AppAuthHandler(consumer_key, consumer_secret)
# auth = tw.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if not api:
    print("Problem connecting to API")


search_words = "Iran" + " -filter:retweets" + " since:2020-01-10" + " until:2020-01-14"
# date_since = "2020-01-02"
# date_till = "2020-01-03"

maxTweets = 100000
tweetsPerQry = 100

tweetCount = 0

relevantPosts = []
with open('tweets_Iran.txt', 'w', encoding="utf-8") as myfile:
    writer = csv.writer(myfile, delimiter='|', quotechar='"', quoting=csv.QUOTE_ALL)

    for tweet in tw.Cursor(api.search, q=search_words, lang="en", tweet_mode='extended',
                           geocode="39.959796,-75.181894,500mi").items(maxTweets):

        tweet_id = tweet.id
        tweet_created_time = tweet.created_at.strftime("%Y-%m-%d %H:%M")
        full_text = tweet.full_text.replace('\n', ' ').replace('\r', '')
        truncated = tweet.truncated
        retweet_count = tweet.retweet_count
        like_count = tweet.favorite_count
        num_of_hashtag = len(tweet.entities['hashtags'])
        num_of_mentions = len(tweet.entities['user_mentions'])
        num_of_url = len(tweet.entities['urls'])
        user_id = tweet.user.id
        screen_name = tweet.user.screen_name
        followers_count = tweet.user.followers_count
        followees_count = tweet.user.friends_count
        account_created_at = tweet.user.created_at.strftime("%Y-%m-%d %H:%M")
        num_of_all_tweets = tweet.user.statuses_count
        verified_user = tweet.user.verified
        scraping_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

        post_tweet = [tweet_id, tweet_created_time, truncated, retweet_count, like_count,
                  num_of_hashtag, num_of_mentions, num_of_url, user_id, screen_name, followers_count, followees_count,
                  account_created_at, num_of_all_tweets, verified_user, scraping_time, full_text]

        # relevantPosts.append(post_tweet)

        writer.writerow(post_tweet)

        tweetCount += 1

        # Display how many tweets we have collected
        print("Downloaded {0} tweets".format(tweetCount))

print(api.rate_limit_status()['resources']['search'])
# print(relevantPosts)

```





```
# This is formatted as code
'''
Gets text content for tweet IDs
'''

# standard
from __future__ import print_function
import getopt
import logging
import os
import sys
# import traceback
# third-party: `pip install tweepy`
import tweepy

# global logger level is configured in main()
Logger = None

# Generate your own at https://apps.twitter.com/app
CONSUMER_KEY = 'hGOrUWoKSViI11fSGNTkoGiHD'
CONSUMER_SECRET = 'gyAHSYn03WlCf6pVmBJcs0wqvXIeCKGv7Q9B28gJGIb5sMulsA'
OAUTH_TOKEN = '1198651355130454016-DeyVvfOmL0FM9Aqk2KDU1c9adlTjYA'
OAUTH_TOKEN_SECRET = 'SDNwpW4iI2L57G59nkfeU8gumc7KuluAXU9svG9xx7vLR'

# batch size depends on Twitter limit, 100 at this time
batch_size=100

def get_tweet_id(line):
    '''
    Extracts and returns tweet ID from a line in the input.
    '''
    (tagid,user_id) = line.split('\t')
    return tweet_id

def get_tweets_single(twapi, idfilepath):
    '''
    Fetches content for tweet IDs in a file one at a time,
    which means a ton of HTTPS requests, so NOT recommended.

    `twapi`: Initialized, authorized API object from Tweepy
    `idfilepath`: Path to file containing IDs
    '''
    # process IDs from the file
    with open(idfilepath, 'r') as idfile:
        lineReader = csv.reader(tsvfile, delimiter="\t")
        for line in idfile[:100]:
            tweet_id = line[0]
            Logger.debug('get_tweets_single: fetching tweet for ID %s', tweet_id)
            try:
                tweet = twapi.get_status(tweet_id)
                print('%s,%s' % (tweet_id, tweet.text.encode('UTF-8')))
            except tweepy.TweepError as te:
                Logger.warn('get_tweets_single: failed to get tweet ID %s: %s', tweet_id, te.message)
                # traceback.print_exc(file=sys.stderr)
        # for
    # with

def get_tweet_list(twapi, idlist):
    '''
    Invokes bulk lookup method.
    Raises an exception if rate limit is exceeded.
    '''
    # fetch as little metadata as possible
    tweets = twapi.statuses_lookup(id_=idlist, include_entities=False, trim_user=True)
    if len(idlist) != len(tweets):
        Logger.warn('get_tweet_list: unexpected response size %d, expected %d', len(tweets), len(idlist))
    for tweet in tweets:
        print('%s,%s' % (tweet.id, tweet.text.encode('UTF-8')))

def get_tweets_bulk(twapi, idfilepath):
    '''
    Fetches content for tweet IDs in a file using bulk request method,
    which vastly reduces number of HTTPS requests compared to above;
    however, it does not warn about IDs that yield no tweet.

    `twapi`: Initialized, authorized API object from Tweepy
    `idfilepath`: Path to file containing IDs
    '''    
    # process IDs from the file
    tweet_ids = list()
    with open(idfilepath, 'r') as idfile:
        lineReader = csv.reader(tsvfile, delimiter="\t")
        for line in idfile[:100]:
            tweet_id = line[0]
            Logger.debug('Enqueing tweet ID %s', tweet_id)
            tweet_ids.append(tweet_id)
            # API limits batch size
            if len(tweet_ids) == batch_size:
                Logger.debug('get_tweets_bulk: fetching batch of size %d', batch_size)
                get_tweet_list(twapi, tweet_ids)
                tweet_ids = list()
    # process remainder
    if len(tweet_ids) > 0:
        Logger.debug('get_tweets_bulk: fetching last batch of size %d', len(tweet_ids))
        get_tweet_list(twapi, tweet_ids)

def usage():
    print('Usage: get_tweets_by_id.py [options] file')
    print('    -s (single) makes one HTTPS request per tweet ID')
    print('    -v (verbose) enables detailed logging')
    sys.exit()

def main():
    # connect to twitter
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    api = tweepy.API(auth)
    with open('en_ids_2020-04-25.tsv') as fp:
      idfile = fp.read()
    get_tweets_bulk(api, idfile)


main()
```



In [ ]:
import os
import tweepy as tw
import csv
import json
import datetime
import csv
import unicodedata
import pandas as pd

In [ ]:
!git clone https://github.com/orkuntemiz/Misinformation_COVID-19.git

Cloning into 'Misinformation_COVID-19'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 114 (delta 49), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (114/114), 6.80 MiB | 5.73 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [ ]:
!ls Misinformation_COVID-19/Data/Dataset-I

Misinformation_COVID-19/Data/Dataset-I


In [ ]:
def get_tweet_id(line):
    '''
    Extracts and returns tweet ID from a line in the input.
    '''
    (tweet_id,user_id) = line.split('\t')
    return tweet_id


Misinformation_COVID19=pd.read_csv('Misinformation_COVID-19/Dataset-I.csv')

In [ ]:
Misinformation_COVID19=Misinformation_COVID19[["tweet_id","tweet_class"]]

In [ ]:
!git clone https://github.com/cuilimeng/CoAID.git

Cloning into 'CoAID'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 83 (delta 23), reused 45 (delta 10), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [ ]:
str(api.get_status('1300187087421886464',tweet_mode="extended").full_text)

'Is this true??\n\nThe #CDC quietly updated the #Covid number to admit that only 6% of all the 153,504 deaths recorded actually died from #COVID-19 .  \n\n153,504 x .06  = 9,210 deaths.  \n\nIs this #True?!?'

In [ ]:
api.get_status('1300187087421886464')

Status(_api=<tweepy.api.API object at 0x7f642e19ec90>, _json={'created_at': 'Sun Aug 30 21:42:12 +0000 2020', 'id': 1300187087421886464, 'id_str': '1300187087421886464', 'text': 'Is this true??\n\nThe #CDC quietly updated the #Covid number to admit that only 6% of all the 153,504 deaths recorded… https://t.co/W4rxLqR5xh', 'truncated': True, 'entities': {'hashtags': [{'text': 'CDC', 'indices': [20, 24]}, {'text': 'Covid', 'indices': [45, 51]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/W4rxLqR5xh', 'expanded_url': 'https://twitter.com/i/web/status/1300187087421886464', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 16078194, 'id_str': '16078194', 'name': 'Scooter McGee',

In [ ]:
api.get_status('1300479046636179456',tweet_mode="extended").entities['urls'][0]['expanded_url']

IndexError: ignored

In [ ]:
#ClaimFake=pd.read_csv('CoAID/09-01-2020/ClaimFakeCOVID-19.csv')
ClaimReal07=pd.read_csv('CoAID/07-01-2020/ClaimRealCOVID-19.csv')
ClaimFake07=pd.read_csv('CoAID/07-01-2020/ClaimFakeCOVID-19.csv')
ClaimReal05=pd.read_csv('CoAID/05-01-2020/ClaimRealCOVID-19.csv')
ClaimFake05=pd.read_csv('CoAID/05-01-2020/ClaimFakeCOVID-19.csv')
ClaimReal09=pd.read_csv('CoAID/09-01-2020/ClaimRealCOVID-19.csv')


NewsFake09=pd.read_csv('CoAID/09-01-2020/NewsFakeCOVID-19.csv')
NewsReal09=pd.read_csv('CoAID/09-01-2020/NewsRealCOVID-19.csv')
NewsFake07=pd.read_csv('CoAID/07-01-2020/NewsFakeCOVID-19.csv')
NewsReal07=pd.read_csv('CoAID/07-01-2020/NewsRealCOVID-19.csv')
NewsFake05=pd.read_csv('CoAID/05-01-2020/NewsFakeCOVID-19.csv')
NewsReal05=pd.read_csv('CoAID/05-01-2020/NewsRealCOVID-19.csv')

In [ ]:
df=pd.concat([ClaimReal07,ClaimFake07,ClaimReal05,ClaimFake05,ClaimReal09],axis=0)

In [ ]:
df

,Unnamed: 0,fact_check_url,news_url,title
0,100166,who.int,https://www.who.int/emergencies/diseases/novel...,People should NOT wear masks while exercising
1,100167,who.int,https://www.who.int/emergencies/diseases/novel...,The likelihood of shoes spreading COVID-19 is ...
2,100168,who.int,https://www.who.int/emergencies/diseases/novel...,The coronavirus disease (COVID-19) is caused b...
3,100169,who.int,https://www.who.int/emergencies/diseases/novel...,The prolonged use of medical masks* when prope...
4,100170,who.int,https://www.who.int/emergencies/diseases/novel...,Most people who get COVID-19 recover from it
...,...,...,...,...
111,100449,who.int,https://www.who.int/emergencies/diseases/novel...,The coronavirus disease (COVID-19) is caused b...
112,100450,who.int,https://www.who.int/emergencies/diseases/novel...,The likelihood of shoes spreading COVID-19 is ...
113,100451,who.int,https://www.who.int/emergencies/diseases/novel...,People should NOT wear masks while exercising
114,100452,who.int,https://www.who.int/emergencies/diseases/novel...,Studies show hydroxychloroquine does not have ...


In [ ]:
import re
regex = r"^Can\s.*"

In [ ]:
series = df["title"]
df1 = series. str. contains(regex)
print(df1)

0      False
1      False
2      False
3      False
4      False
       ...  
111    False
112    False
113    False
114    False
115    False
Name: title, Length: 482, dtype: bool


In [ ]:
df1.name=['Q']

In [ ]:
df_conc=pd.concat([df1,df],axis=1)

In [ ]:
df_claims=df_conc[df_conc.Q==False]

In [ ]:
df_qa=df_conc[df_conc.Q==True]

In [ ]:
series = df_qa["title"]
df2 = series. str. contains(regex)
print(df2)

12     False
13     False
14     False
15     False
16     False
       ...  
102    False
103    False
104    False
105    False
106    False
Name: title, Length: 266, dtype: bool


In [ ]:
df2.name='CAN'

In [ ]:
df_conc2=pd.concat([df2,df_qa],axis=1)

In [ ]:
df_can=df_conc2[df_conc2.CAN==True]

In [ ]:
df_can.title=df_can.title.str.replace('Can ','')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df_can.title=df_can.title.str.replace('?','')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df_claims_all=pd.concat([df_can,df_claims],axis=0)

In [ ]:
df_claims_all=df_claims_all.reset_index()

In [ ]:
df_claims

,Q,Unnamed: 0,fact_check_url,news_url,title
0,False,100166,who.int,https://www.who.int/emergencies/diseases/novel...,People should NOT wear masks while exercising
1,False,100167,who.int,https://www.who.int/emergencies/diseases/novel...,The likelihood of shoes spreading COVID-19 is ...
2,False,100168,who.int,https://www.who.int/emergencies/diseases/novel...,The coronavirus disease (COVID-19) is caused b...
3,False,100169,who.int,https://www.who.int/emergencies/diseases/novel...,The prolonged use of medical masks* when prope...
4,False,100170,who.int,https://www.who.int/emergencies/diseases/novel...,Most people who get COVID-19 recover from it
...,...,...,...,...,...
111,False,100449,who.int,https://www.who.int/emergencies/diseases/novel...,The coronavirus disease (COVID-19) is caused b...
112,False,100450,who.int,https://www.who.int/emergencies/diseases/novel...,The likelihood of shoes spreading COVID-19 is ...
113,False,100451,who.int,https://www.who.int/emergencies/diseases/novel...,People should NOT wear masks while exercising
114,False,100452,who.int,https://www.who.int/emergencies/diseases/novel...,Studies show hydroxychloroquine does not have ...


In [ ]:
#This code creates the dataset from Corpus.csv which is downloadable from the
#internet well known dataset which is labeled manually by hand. But for the text
#of tweets you need to fetch them with their IDs.
import tweepy

# Twitter Developer keys here
# It is CENSORED
consumer_key = 'hGOrUWoKSViI11fSGNTkoGiHD'
consumer_key_secret = 'gyAHSYn03WlCf6pVmBJcs0wqvXIeCKGv7Q9B28gJGIb5sMulsA'
access_token = '1198651355130454016-DeyVvfOmL0FM9Aqk2KDU1c9adlTjYA'
access_token_secret = 'SDNwpW4iI2L57G59nkfeU8gumc7KuluAXU9svG9xx7vLR'

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# This method creates the training set
def createTrainingSet(corpusFile, targetResultFile):
    import csv
    import time

    counter = 0
    corpus = []

    with open(corpusFile, 'r') as csvfile:
        lineReader = csv.reader(csvfile, delimiter="," , quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id": row[0], "tweet_class": row[1]})

    sleepTime = 1
    trainingDataSet = []

    for tweet in corpus:
        try:
            tweetFetched = api.get_status(tweet["tweet_id"],tweet_mode="extended")
            print("Tweet fetched" + tweetFetched.full_text)
            tweet["text"] = tweetFetched.full_text
            tweet["lang"] = tweetFetched.lang
            tweet["created_at"]=str(tweetFetched.created_at.strftime('%m.%d.%Y'))
            #tweet["expanded_url"]=tweetFetched.entities['urls'][0]['expanded_url']
            trainingDataSet.append(tweet)
            time.sleep(sleepTime)
        except:
            print("Inside the exception - no:2")

            continue

    with open(targetResultFile, 'w',encoding='utf-8-sig') as csvfile:
        linewriter = csv.writer(csvfile, delimiter=',', quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"],tweet["created_at"],tweet["lang"],tweet["tweet_class"]])
            except Exception as e:
                print(e)
                print("Inside the exception - no:3")
    return trainingDataSet



In [ ]:
for tweet in corpus[4:6]:


ValueError: ignored

In [ ]:
if 1==1: 
    counter = 0
    corpus = []

    with open('Misinformation_COVID-19/Dataset-I.csv', 'r') as csvfile:
        lineReader = csv.reader(csvfile, delimiter="," , quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id": row[0], "tweet_class": row[1]})

    sleepTime = 1
    trainingDataSet = []

In [ ]:
corpus[1]

{'tweet_class': 'FALSE', 'tweet_id': '1.28664E+18'}

In [ ]:
api.get_status(tweet["tweet_id"],tweet_mode="extended")

In [ ]:
# Code starts here
# This is corpus dataset
corpusFile = 'Misinformation_COVID-19/Dataset-I.csv'
# This is my target file
targetResultFile = "Dataset-I.csv"
# Call the method
resultFileFakeClaims = createTrainingSet(corpusFile, targetResultFile)

from google.colab import files
files.download("Dataset-I.csv")

from google.colab import files
import json

x = json.dumps (resultFileFakeClaims)
y = x.encode('utf-8')
with open('Dataset-I.txt', 'w') as f:
  f.write(str(y))

files.download('Dataset-I.txt')

Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Code starts here
# This is corpus dataset
corpusFile = 'CoAID/05-01-2020/ClaimRealCOVID-19_tweets.csv'
# This is my target file
targetResultFile = "ClaimRealCOVID-19TweetsV10.csv"
# Call the method
resultFileRealClaims = createTrainingSet(corpusFile, targetResultFile)

from google.colab import files
files.download("ClaimRealCOVID-19TweetsV10.csv")
from google.colab import files
import json

x = json.dumps (resultFileRealClaims)
y = x.encode('utf-8')
with open('resultFileRealClaimsV10.txt', 'w') as f:
  f.write(str(y))

files.download('resultFileRealClaimsV10.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Code starts here
# This is corpus dataset
corpusFile = 'CoAID/05-01-2020/NewsFakeCOVID-19_tweets.csv'
# This is my target file
targetResultFile = "NewsFakeCOVID-19TweetsV10.csv"
# Call the method
resultFileFakeNews = createTrainingSet(corpusFile, targetResultFile)

from google.colab import files
files.download("NewsFakeCOVID-19TweetsV10.csv")
from google.colab import files
import json

x = json.dumps (resultFileFakeNews)
y = x.encode('utf-8')
with open('resultFileFakeNewsV10.txt', 'w') as f:
  f.write(str(y))

files.download('resultFileFakeNewsV10.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("NewsFakeCOVID-19TweetsV10.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:


corpusFile = 'CoAID/05-01-2020/NewsRealCOVID-19_tweets.csv'
# This is my target file
targetResultFile = "NewsRealCOVID-19TweetsV10.csv"
# Call the method
resultFileRealNews = createTrainingSet(corpusFile, targetResultFile)

from google.colab import files
files.download("NewsRealCOVID-19TweetsV10.csv")
from google.colab import files
import json

x = json.dumps (resultFileRealNews)
y = x.encode('utf-8')
with open('resultFileRealNewsV10.txt', 'w') as f:
  f.write(str(y))

files.download('resultFileRealNewsV10.txt')

In [ ]:
from google.colab import files
files.download("NewsRealCOVID-19TweetsV11_1.csv")
from google.colab import files
import json

x = json.dumps (resultFileRealNews)
y = x.encode('utf-8')
with open('resultFileRealNewsV11_1.txt', 'w') as f:
  f.write(str(y))

files.download('resultFileRealNewsV11_1.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth #You may upload files to your drive and give authentication in order to read the csv.I read files from drive without any problem. 
from pydrive.drive import GoogleDrive #I hope when you run them for grading purpose you won't encounter any problem
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

In [ ]:
data_downloaded = drive.CreateFile({'id': '1usPHk3ksoIF_FheuLJ8cyfmYCbJz20l4'})
data_downloaded.GetContentFile('train_df.csv')
CovidClaims = pd.read_csv('train_df.csv')

In [ ]:
import requests

headers = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
    'x-api-key': 'bd4bc8ac0c914cd4ac0e61697bbab31c',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': '',
    'Accept-Language': '*',
}

In [ ]:
CovidClaims=CovidClaims[CovidClaims.DATASET=='CLAIM']

In [ ]:
CovidClaims

,index,FULL_NEWS,ID,DATE,LANG,INDEX,TRUTH,DATASET,hashtags,mentions,links
0,29,"BlackFriday with commentary provided by , , ...",1.200463e+18,2019-11-29,en,1,False,CLAIM,BlackFriday,JRsBBQ ShutUpExcalibur tonyschiavone24,https://t.co/KvCNMcYFFw
45,78,Day 1⃣1⃣: In this series Dr. Nicole Culos-Reed...,1.207410e+18,2019-12-18,en,100066,True,CLAIM,no,no,https://t.co/N48w9eR5kj https://t.co/Q8YYzGOs1P
69,102,What is physical activity? The World Health ...,1.209110e+18,2019-12-23,en,100066,True,CLAIM,no,no,https://t.co/wMt58pg1OI
145,179,What are the International Health Regulations ...,1.216830e+18,2020-01-13,en,100134,True,CLAIM,no,no,https://t.co/UID9a48795
147,181,Preliminary investigations conducted by the Ch...,1.217043e+18,2020-01-14,en,1,False,CLAIM,coronavirus Wuhan China,no,https://t.co/Fnl5P877VG
...,...,...,...,...,...,...,...,...,...,...,...
56414,58267,What is situation with potential COVID19 expos...,1.299160e+18,2020-08-28,en,100348,True,CLAIM,COVID19,no,no
56417,58270,Then someone will make remarks of why you even...,1.300020e+18,2020-08-30,en,100415,True,CLAIM,no,no,no
56419,58272,hopefully the kpop industry doesn’t completely...,1.300080e+18,2020-08-30,en,100415,True,CLAIM,no,no,no
56422,58275,I have here a LONG list crowd sourced by our m...,1.300400e+18,2020-08-31,en,100338,True,CLAIM,no,BorisJohnson,no


In [ ]:
checkWorthyClaim=CheckWorthynessControl('Day 1⃣1⃣:')

In [ ]:
def CheckWorthynessControl(input_tweet): 
  #input_tweet='As we have mentioned above, COVID-19 is only fatal for a small percentage of people. In fact 80% face only mild symptoms with no need for hospitalization.'
  input_tweet=requests.compat.quote_plus(input_tweet)
  api_url = "https://idir.uta.edu/claimbuster/api/v2/bba/score/text/{input_text}".format(input_text=input_tweet)
  response =  requests.get(api_url, headers=headers)
  results=response.json()['results']
  string_all=''
  for i in range(len(results)):
    if results[i]['result']=='Check-worthy factual statement':
      string_all=string_all+' '+results[i]['text']
  return string_all

In [ ]:
checkWorthyClaims_df=[]
for inx, claim in CovidClaims.iterrows():
  try:
    checkWorthyClaim=CheckWorthynessControl(claim['FULL_NEWS'])
    if checkWorthyClaim != '' :
      claim['checkWorthyText']=checkWorthyClaim
      checkWorthyClaims_df.append(claim)
      print(claim)
  except: 
    continue

Streaming output truncated to the last 5000 lines.
INDEX                                                              1
TRUTH                                                          False
DATASET                                                        CLAIM
hashtags                                                          no
mentions                                                 assampolice
links                                        https://t.co/zcO8Al3nQX
checkWorthyText     Fake News Alert : Video is being circulated t...
Name: 28407, dtype: object
index                                                          29878
FULL_NEWS          These people have been lying about my record s...
ID                                                         1.265e+18
DATE                                                      2020-05-25
LANG                                                              en
INDEX                                                              1
TRUTH                    

In [ ]:
from google.colab import files

In [ ]:
with open(checkWorthyClaims_df, 'w',encoding='utf-8-sig') as csvfile:
  from google.colab import files
  files.download(csvfile)

TypeError: ignored

In [ ]:
from google.colab import files
files.download("NewsRealCOVID-19TweetsV11_1.csv")

In [ ]:
csvfile=pd.DataFrame(checkWorthyClaims_df).to_csv(index=False)
files.download(csvfile)

In [ ]:
ClaimsCW_df=pd.DataFrame(checkWorthyClaims_df)

In [ ]:
from google.colab import files

In [ ]:
df.to_csv('Claims_Raw.csv', encoding='utf-8-sig')

In [ ]:
files.download('Claims_Raw.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import time
import nltk
import requests
from nltk.stem import *
stemmer = SnowballStemmer("english")

In [ ]:
stemmer.stem('masks')

'mask'

In [ ]:
!pip install spacy
import spacy
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

In [ ]:
!python -m spacy download en_core_web_sm
!pip install cov-bsv
import cov_bsv

nlp = cov_bsv.load()
doc = nlp('Pt tested for COVID-19. His wife was recently diagnosed with novel coronavirus. SARS-COV-2: Detected')

In [ ]:
doc=nlp("Apple is looking at buying U.K. startup for $1 billion covid-19 testing")
print([(w.text, w.pos_) for w in doc])

[('Apple', 'PROPN'), ('is', 'AUX'), ('looking', 'VERB'), ('at', 'ADP'), ('buying', 'VERB'), ('U.K.', 'PROPN'), ('startup', 'NOUN'), ('for', 'ADP'), ('$', 'SYM'), ('1', 'NUM'), ('billion', 'NUM'), ('covid-19', 'NOUN'), ('testing', 'NOUN')]


In [ ]:
doc.ents

(Apple, U.K., $1 billion)